In [42]:
!pip install mediapipe
!pip install gradio
!pip install opencv-python
!pip install matplotlib

/Users/bilbaothanos14/.zshenv:export:1: not valid in this context: /Library/Java/JavaVirtualMachines/jdk-19.jdk/Contents/Home
/Users/bilbaothanos14/.zshenv:export:1: not valid in this context: /Library/Java/JavaVirtualMachines/jdk-19.jdk/Contents/Home


/Users/bilbaothanos14/.zshenv:export:1: not valid in this context: /Library/Java/JavaVirtualMachines/jdk-19.jdk/Contents/Home
/Users/bilbaothanos14/.zshenv:export:1: not valid in this context: /Library/Java/JavaVirtualMachines/jdk-19.jdk/Contents/Home


In [41]:
import gradio as gr
import mediapipe as mp
import numpy as np
import cv2

mp_selfie = mp.solutions.selfie_segmentation

def segment(image):
    with mp_selfie.SelfieSegmentation(model_selection=0) as model:
        res = model.process(image)
        mask = np.stack((res.segmentation_mask,)*3, axis=-1) > 0.5
        return np.where(mask, image, cv2.blur(image, (40, 40)))
#the code above is just a placeholder model I pulled off the internet: this can be replaced with our actual ASL model later


webcam = gr.inputs.Image(shape=(640, 480), source="webcam")


output_image = gr.outputs.Image(type="pil")

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Tabs() as tabs:
        with gr.TabItem("Learning Mode", id=0):
            t = gr.Textbox()
        with gr.TabItem("Game Mode", id=1):
            i = gr.Interface(fn=segment, inputs=webcam, outputs=output_image, live=True)
            
demo.launch()

/var/folders/mf/ynycfx8d0fggrrcjj0y677dc0000gn/T/ipykernel_2047/3726164489.py:16: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  webcam = gr.inputs.Image(shape=(640, 480), source="webcam")
/var/folders/mf/ynycfx8d0fggrrcjj0y677dc0000gn/T/ipykernel_2047/3726164489.py:16: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  webcam = gr.inputs.Image(shape=(640, 480), source="webcam")
/var/folders/mf/ynycfx8d0fggrrcjj0y677dc0000gn/T/ipykernel_2047/3726164489.py:19: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output_image = gr.outputs.Image(type="pil")


Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/bilbaothanos14/miniforge3/lib/python3.9/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/Users/bilbaothanos14/miniforge3/lib/python3.9/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/bilbaothanos14/miniforge3/lib/python3.9/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/Users/bilbaothanos14/miniforge3/lib/python3.9/site-packages/gradio/blocks.py", line 1185, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/bilbaothanos14/miniforge3/lib/python3.9/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/bilbaothanos14/miniforge3/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    retur